## Task 1 - SQL

In [1]:
from google.colab import drive 
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 57.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=3614387e5bd936675e9ea151c4d78390043d945e371bbdc86ae56f8373727ef6
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


### Build SparkSession:

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

### Read the json file:

In [54]:
df = spark.read.json('/content/drive/MyDrive/DataFrames_sample.json')

In [55]:
df.show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
| 8.0|20.3|  1TB SSD|  4|       iMac|64GB|       27"| 25.6|  20.8|2017|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Display the schema:


In [56]:
df.printSchema()

root
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- HDD: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- W: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Year: long (nullable = true)



### Get all the data when "Model" equal "MacBook Pro":




In [7]:
# df.select(df['name']).show()
df.filter(df.Model == "MacBook Pro").show(truncate=False)

+----+----+---------+---+-----------+----+----------+-----+------+----+
|D   |H   |HDD      |Id |Model      |RAM |ScreenSize|W    |Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|1  |MacBook Pro|16GB|15"       |13.75|4.02  |2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



In [63]:
df.createOrReplaceTempView('DF_view')
spark.sql("""SELECT * FROM DF_view where Model='MacBook Pro'""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Create Table:

### Display "RAM"column and count "RAM" column:

In [8]:
df.groupBy('RAM').count().show()

+----+-----+
| RAM|count|
+----+-----+
|64GB|    1|
|16GB|    1|
| 8GB|    2|
+----+-----+



In [69]:
spark.sql(""" SELECT RAM , COUNT(RAM) FROM DF_view group by (RAM)  """).show()

+----+----------+
| RAM|count(RAM)|
+----+----------+
|64GB|         1|
|16GB|         1|
| 8GB|         2|
+----+----------+



### Get all columns when "Year" column equal "2015"  

In [9]:
from pyspark.sql.functions import *

In [10]:
df.filter(col('Year')==2015).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



In [70]:
spark.sql(""" SELECT * FROM DF_view where Year=2015  """).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all when "Model" start with "M":

In [11]:
df.filter(df.Model.startswith('M')).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+



In [72]:
spark.sql(""" SELECT * FROM DF_view where Model like 'M%'  """).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data when "Model" column equal "MacBook Pro"

In [73]:
spark.sql(""" SELECT * FROM DF_view where Model='MacBook Pro' """).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [12]:
df.filter((col('RAM') == '8GB') & (col('Model') == 'MacBook')).show()


+----+----+---------+---+-------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|  Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|MacBook|8GB|       12"|11.04|  2.03|2016|
+----+----+---------+---+-------+---+----------+-----+------+----+



In [74]:
spark.sql(""" SELECT * FROM DF_view where RAM='8GB' and Model='MacBook' """).show()

+----+----+---------+---+-------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|  Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|MacBook|8GB|       12"|11.04|  2.03|2016|
+----+----+---------+---+-------+---+----------+-----+------+----+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [13]:
df.filter((col('D') >= '8') & (col('Model') == 'iMac')).show()

+---+----+-------+---+-----+----+----------+----+------+----+
|  D|   H|    HDD| Id|Model| RAM|ScreenSize|   W|Weight|Year|
+---+----+-------+---+-----+----+----------+----+------+----+
|8.0|20.3|1TB SSD|  4| iMac|64GB|       27"|25.6|  20.8|2017|
+---+----+-------+---+-----+----+----------+----+------+----+



In [75]:
spark.sql(""" SELECT * FROM DF_view where D>='8' and Model='iMac' """).show()

+---+----+-------+---+-----+----+----------+----+------+----+
|  D|   H|    HDD| Id|Model| RAM|ScreenSize|   W|Weight|Year|
+---+----+-------+---+-----+----+----------+----+------+----+
|8.0|20.3|1TB SSD|  4| iMac|64GB|       27"|25.6|  20.8|2017|
+---+----+-------+---+-----+----+----------+----+------+----+



## Task 2


### Read "test1" dataset:

In [14]:
df = spark.read.csv('/content/drive/MyDrive/test1.csv',header=True,inferSchema=True)

In [15]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



### Display Salary of the people less than or equal to 20000

In [16]:
df.filter(col('Salary')<=20000).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [17]:
df.select('Name','Salary').where(df.Salary <= 20000).show()

+-------+------+
|   Name|Salary|
+-------+------+
|  Sunny| 20000|
|   Paul| 20000|
| Harsha| 15000|
|Shubham| 18000|
+-------+------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [18]:
df.select(df.Name, df.Salary.between(15000, 20000)).show()

+---------+-----------------------------------------+
|     Name|((Salary >= 15000) AND (Salary <= 20000))|
+---------+-----------------------------------------+
|    Krish|                                    false|
|Sudhanshu|                                    false|
|    Sunny|                                     true|
|     Paul|                                     true|
|   Harsha|                                     true|
|  Shubham|                                     true|
+---------+-----------------------------------------+



In [19]:
d=df.filter((col('Salary') >= '15000') & (col('Salary') <= '20000'))
d.select(col('Name'),col('Salary')).show()

+-------+------+
|   Name|Salary|
+-------+------+
|  Sunny| 20000|
|   Paul| 20000|
| Harsha| 15000|
|Shubham| 18000|
+-------+------+



## Task 3 

### Read "test3" dataset:

In [20]:
df = spark.read.csv('/content/drive/MyDrive/test3.csv',header=True,inferSchema=True)

### Display dataset

In [21]:
df.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



### Display schema

In [22]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



### Group by "Name" column and using sum function on "Name" column

In [23]:
df.groupBy('Name').agg(count('Name').alias('count_name')).show() ##### not sure 

+---------+----------+
|     Name|count_name|
+---------+----------+
|Sudhanshu|         3|
|    Sunny|         2|
|    Krish|         3|
|   Mahesh|         2|
+---------+----------+



In [24]:
df.groupBy("Name").sum("salary").show(truncate=False)

+---------+-----------+
|Name     |sum(salary)|
+---------+-----------+
|Sudhanshu|35000      |
|Sunny    |12000      |
|Krish    |19000      |
|Mahesh   |7000       |
+---------+-----------+



### Group by "Name" column and using avg function on "Name" column

In [25]:
df.groupBy("Name").avg("salary").show(truncate=False)

+---------+------------------+
|Name     |avg(salary)       |
+---------+------------------+
|Sudhanshu|11666.666666666666|
|Sunny    |6000.0            |
|Krish    |6333.333333333333 |
|Mahesh   |3500.0            |
+---------+------------------+



### Group by "Departments" column and using sum function on "Departments" column

In [26]:
df.groupBy("Departments").sum("salary").show(truncate=False)

+------------+-----------+
|Departments |sum(salary)|
+------------+-----------+
|IOT         |15000      |
|Big Data    |15000      |
|Data Science|43000      |
+------------+-----------+



### Group by "Departments" column and using mean function on "Departments" column:

In [27]:
df.groupBy("Departments").avg("salary").show(truncate=False)

+------------+-----------+
|Departments |avg(salary)|
+------------+-----------+
|IOT         |7500.0     |
|Big Data    |3750.0     |
|Data Science|10750.0    |
+------------+-----------+



Group by "Departments" column and using count function on "Departments" column:

In [28]:
# df.groupBy("Departments").count("Departments").show(truncate=False)
df.groupBy('Departments').agg(count('Departments').alias('count_dep')).show()

+------------+---------+
| Departments|count_dep|
+------------+---------+
|         IOT|        2|
|    Big Data|        4|
|Data Science|        4|
+------------+---------+



### Apply agg to using sum function get the total of salaries

In [29]:
df.groupBy('Name').agg(sum('salary').alias('sum_salary')).show()

+---------+----------+
|     Name|sum_salary|
+---------+----------+
|Sudhanshu|     35000|
|    Sunny|     12000|
|    Krish|     19000|
|   Mahesh|      7000|
+---------+----------+



## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [95]:
df = spark.read.csv('/content/drive/MyDrive/ITI_data.csv',header=True,inferSchema=True)

In [96]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [97]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [98]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     null|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

### OneHotEncoder 


In [125]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler

In [126]:
categoricalCols = [f for (f,d) in df.dtypes if d=='string' and f!='Ship_name']

In [127]:
categoricalCols

['Cruise_line']

In [128]:
indexOutputCols = [s + "_Index" for s in categoricalCols]

In [129]:
indexOutputCols

['Cruise_line_Index']

In [130]:
oheOutputCols = [s + "_OHE" for s in categoricalCols]

In [131]:
oheOutputCols

['Cruise_line_OHE']

In [132]:
stringIndexer = StringIndexer(inputCols=categoricalCols,outputCols=indexOutputCols,handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)

In [133]:
oheEncoder

OneHotEncoder_dc18ef1c4425

In [134]:
numericCols = [f for (f,d) in df.dtypes if ((d == 'double')&(f!='crew'))]

In [135]:
numericCols

['Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']

###Use VectorAssembler to merge all columns into one column:

In [136]:
assemblerInputs = oheOutputCols + numericCols

In [137]:
assemblerInputs

['Cruise_line_OHE',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density']

In [138]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

### Create a Linear Regression Model 

In [139]:
trainDF, testDF = df.randomSplit([.8,.2],seed=42)
print(f"There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set")

There are 133 rows in the training set, and 25 in the test set


In [140]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features',
                      labelCol='crew',
                      predictionCol='prediction')

### Creating a Pipeline

In [141]:
from pyspark.ml import Pipeline

In [142]:
myStages = [stringIndexer,oheEncoder,vecAssembler,lr]

In [143]:
pipeline = Pipeline(stages=myStages)

In [144]:
pipelineModel = pipeline.fit(trainDF)

In [145]:
predDF = pipelineModel.transform(testDF)

In [146]:
from pyspark.ml.evaluation import RegressionEvaluator

In [147]:
regeval_r2 = RegressionEvaluator(predictionCol='prediction',labelCol='crew',metricName='r2')

In [148]:
regeval_r2.evaluate(predDF)

0.9728301900481229

By Eng. Mostafa Nabieh 
If you have questions, please feel free to ask.

My Email : nabieh.mostafa@yahoo.com

My Whatsapp : +201015197566